# Text Tokenizer

자연어처리에 있어 토크나이저는 문장을 작은 단위의 토큰(단어 또는 부분단어)으로 분할하는 역할을 하며, 
본 실습에서는 대표적인 SubWord Tokenizer 방법들에 대해 살펴보도록 하겠습니다.  
- BPE Tokenization (Algorithm) 
- Pretrained Tokenizer (WordPiece)  
- Tokenizer Training 

## 0. Setup

In [1]:
import os
os.environ['HF_HOME'] = 'D:/HF/cache'
os.environ['HF_DATASETS'] = 'D:/HF/datasets'
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = "1"

In [ ]:
# !pip install --user transformers==4.51.1

In [2]:
from IPython.display import display
from tqdm.notebook import tqdm as notebook_tqdm

In [3]:
!nvidia-smi

Mon Aug 18 10:55:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 553.62                 Driver Version: 553.62         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q               WDDM  |   00000002:00:00.0 Off |                    0 |
| N/A    0C    P8             N/A /  N/A  |    1110MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 1. BPE Tokenization (Algorithm)

BPE 토크나이저는 기본 단위(단일 문자)의 리스트로 시작해서 가장 빈번하게 함께 등장하는 토큰을 합쳐 어휘사전에 새롭게 추가하는 방식으로 원하는 어휘사전 크기에 도달할 때까지 점진적으로 새 토큰 만드는 과정을 반복합니다.  
아래 실습에서는 논문(https://arxiv.org/pdf/1508.07909.pdf) 에 언급된 알고리즘을 그대로 구현해 보겠습니다.

In [4]:
import re, collections
from IPython.display import display, Markdown, Latex

In [5]:
num_merges = 10

In [6]:
vocab = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>' : 6,
         'w i d e s t </w>' : 3
         }

In [7]:
def get_stats(vocab):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    print('Frequences :', dict(pairs))
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

bpe_codes = {}
bpe_codes_reverse = {}

for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i + 1)))
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print("New merge: {}".format(best))
    print("Vocabulary: {}".format(vocab))

### Iteration 1

Frequences : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
New merge: ('e', 's')
Vocabulary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

Frequences : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
New merge: ('es', 't')
Vocabulary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

Frequences : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('new', 'est</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
New merge: ('new', 'est</w>')
Vocabulary: {'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 9

Frequences : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
New merge: ('low', '</w>')
Vocabulary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

Frequences : {('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
New merge: ('w', 'i')
Vocabulary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


## 2. Pretrained Tokenizer (WordPiece)

트랜스포머스는 사전 훈련된 언어모델을 활용하기 위하여 언어모델 구축에 사용된 토크나이저를 로딩하는 **AutoTokenizer** 클래스를 
제공합니다.  
이 클래스의 **.from_pretrained()** 메소드를 이용하여 모델ID나 로컬 파일 경로로 호출하면 됩니다.  
대표적인 Sub-word Tokenizer 방식으로 WordPiece 계열의 **BERT Tokenizer**를 이용하여 실습해 보겠습니다.

In [8]:
from transformers import AutoTokenizer

# [실습] 다음 코드를 완성하세요!!
# 'distilbert-base-uncased' 모델에 사용된 토크나이저를 불러옵니다.
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer.tokenize("Time flies like an arrow!")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

['time', 'flies', 'like', 'an', 'arrow', '!']

In [9]:
print(tokenizer.vocab_size)
print(tokenizer.model_max_length)
print(tokenizer.model_input_names)

30522
512
['input_ids', 'attention_mask']


In [10]:
sentence = "Tokenizing a text is splitting it into words or subwords."

# [실습] 다음 코드를 완성하세요!!
# tokenizer를 이용하여 sentence를 토크나이즈합니다.
tokenized_sentence = tokenizer.tokenize(sentence)
print(tokenized_sentence)

['token', '##izing', 'a', 'text', 'is', 'splitting', 'it', 'into', 'words', 'or', 'sub', '##words', '.']


In [11]:
inputs = tokenizer(sentence)

encoded_sentence = inputs["input_ids"]
print(encoded_sentence)

[101, 19204, 6026, 1037, 3793, 2003, 14541, 2009, 2046, 2616, 2030, 4942, 22104, 1012, 102]


In [12]:
# [실습] 다음 코드를 완성하세요!!
# tokenizer를 이용하여 encoded_sentence를 문장으로 복원합니다.
decoded_sentence = tokenizer.decode(encoded_sentence)
print(decoded_sentence)

[CLS] tokenizing a text is splitting it into words or subwords. [SEP]


In [13]:
sentence_a = "Time flies like an arrow."
sentence_b = "This sentence is at least longer than the sentence A."

padded_sentence = tokenizer([sentence_a, sentence_b], padding=True)
print(padded_sentence["input_ids"])

[[101, 2051, 10029, 2066, 2019, 8612, 1012, 102, 0, 0, 0, 0, 0], [101, 2023, 6251, 2003, 2012, 2560, 2936, 2084, 1996, 6251, 1037, 1012, 102]]


In [14]:
padded_sentence["attention_mask"]

[[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

## 3. Tokenizer Training

특정 언어 모델이 당신이 원하는 언어를 제대로 지원하지 못하거나, 현재 보유한 코퍼스가 언어 모델과 특성이 다른 경우, 해당 코퍼스에 적합하게 적응된 토크나이저를 이용하여 모델을 새롭게 학습하기를 원할 수도 있습니다. 이를 위해서는 그 데이터셋에 대한 새로운 토크나이저를 학습해야 합니다. 예로, 파이썬 코드를 토큰화 하기 위해 BPE 토크나이저(GPT2-XL)를 재훈련해 보도록 하겠습니다.  

In [15]:
from transformers import AutoTokenizer

model_ckpt = "gpt2-xl"

old_tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [16]:
print(f"Vocabulary Size: {len(old_tokenizer)}")

Vocabulary Size: 50257


Python Code 이용하여 GPT2-XL 토크나이저를 테스트해 보겠습니다.  
'Ġ', 'Ċ' 와 같은 문자가 있는 것은 GPT-2 토크나이저가 유니코드 문자가 아닌 바이트 단위로 동작하기 때문입니다.
('Ġ': Space, 'Ċ': Line Feed,...)

In [17]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."',
 '""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [18]:
import keyword

for keyw in keyword.kwlist:
    if keyw not in old_tokenizer.vocab:
        print(f'Keyword `{keyw}` does not exist in Vocabulary!')

Keyword `async` does not exist in Vocabulary!
Keyword `await` does not exist in Vocabulary!
Keyword `elif` does not exist in Vocabulary!
Keyword `finally` does not exist in Vocabulary!
Keyword `nonlocal` does not exist in Vocabulary!
Keyword `yield` does not exist in Vocabulary!


파이썬 코드 데이터(code_search_net/python)을 이용하여 토크나이저를 새로 학습하겠습니다.

데이터셋 **"code search_net"** 는 CodeSearchNet Challenge를 위해 구축된 것으로, GitHub 상의 오픈소스 코드를 포함하고 있으며 파이썬을 포함 몇개 프로그래밍 언어로 구분되어 있습니다.

In [19]:
from datasets import load_dataset

raw_datasets = load_dataset("//swschoolavdazfiles002.file.core.windows.net/aias-language/Dataset/code_search_net", "python", trust_remote_code=True)

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [20]:
raw_datasets["train"]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [32]:
print(raw_datasets["train"][123455]["func_name"])

doapi.request


In [21]:
print(raw_datasets["train"][123456]["whole_func_string"])

def last_rate_limit(self):
        """
        A `dict` of the rate limit information returned in the most recent
        response, or `None` if no requests have been made yet.  The `dict`
        consists of all headers whose names begin with ``"RateLimit"`` (case
        insensitive).

        The DigitalOcean API specifies the following rate limit headers:

        :var string RateLimit-Limit: the number of requests that can be made
            per hour
        :var string RateLimit-Remaining: the number of requests remaining until
            the limit is reached
        :var string RateLimit-Reset: the Unix timestamp for the time when the
            oldest request will expire from rate limit consideration
        """
        if self.last_response is None:
            return None
        else:
            return {k:v for k,v in iteritems(self.last_response.headers)
                        if k.lower().startswith('ratelimit')}


In [34]:
len(raw_datasets["train"])

412178

In [22]:
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]

In [23]:
training_corpus = get_training_corpus()

기존 토크나이저와 동일한 특성을 가진 새로운 토크나이저를 학습하기 위해 ``AutoTokenizer.train_new_from_iterator()``를 사용하겠습니다.

- 목표하는 Vocabulary Size 설정
- 토크나이저 학습을 위해 입력 문자열을 공급하는 Iterator 설정
- train_new_from_iterator() 메소드 호출

토크나이저 학습에 약 2~3분 정도 소요됩니다.

In [24]:
# [실습] 다음 코드를 완성하세요!!
# training_corpus를 이용하여 토크나이저를 학습합니다.
new_tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [25]:
tokens = new_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'numbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'ĊĠĠĠ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`."""',
 'ĊĠĠĠ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [26]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

27
36


In [27]:
import keyword

for keyw in keyword.kwlist:
    if keyw not in new_tokenizer.vocab:
        print(f'Keyword `{keyw}` does not exist in Vocabulary!')

Keyword `nonlocal` does not exist in Vocabulary!


In [28]:
print(f"Vocabulary Size: {len(new_tokenizer)}")

Vocabulary Size: 52000


- Ref. https://huggingface.co/learn/nlp-course/chapter6